In [18]:
import pandas as pd
import numpy as np
from datetime import date

In [36]:
read_existing_data = pd.read_csv(r'C:\Users\40100171\OneDrive - Anheuser-Busch InBev\Desktop\Technical_Analysis_automated\data\raw\stock_data_raw.csv.gz', compression='gzip')

In [9]:
stock_universe = read_existing_data['Stock Name'].unique().tolist()

In [25]:
start_date = pd.to_datetime(read_existing_data.loc[read_existing_data['Stock Name']==stock_universe[0],'Date'].max())

In [26]:
start_date

Timestamp('2024-08-14 00:00:00')

In [28]:
check = pd.to_datetime(read_existing_data.loc[read_existing_data['Stock Name']==stock_universe[0],'Date'])
delta = check[1] - check[0]

In [38]:
def detect_frequency(date_df):
    date_diffs = date_df.diff().dropna()
    median_diff = date_diffs.median()
    
    daily_threshold = pd.Timedelta(days=1.5)
    weekly_threshold = pd.Timedelta(weeks=1.5)
    
    # Determine frequency based on the median difference
    if median_diff <= daily_threshold:
        return('1d')
    elif median_diff <= weekly_threshold:
        return('1wk')
    else:
        return('1mo')

In [7]:
def adding_incremental_data()
    
    existing_data = pd.read_csv(r'C:\Users\40100171\OneDrive - Anheuser-Busch InBev\Desktop\Technical_Analysis_automated\data\raw\stock_data_raw.csv.gz', compression='gzip')
    stock_universe = read_existing_data['Stock Name'].unique().tolist()
    dates = pd.to_datetime(read_existing_data.loc[read_existing_data['Stock Name']==stock_universe[0],'Date'])
    start_date = dates.max() 
    end_date = date.today()
    interval = detect_frequency(dates)
    full_data = pd.DataFrame()
    
    if(start_date == end_date):
        file
    
    for i in stock_universe:
        df = data_puller(i, start_date, end_date, interval)
        existing_data = pd.concat([existing_data, df])
    
    existing_data.to_csv(r'C:\Users\40100171\OneDrive - Anheuser-Busch InBev\Desktop\Technical_Analysis_automated\data\raw\stock_data_raw.csv.gz', index=False, compression='gzip')
    

KeyError: 'Stock name'